In [3]:
#imports
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np
import os, io
from surprise import KNNBasic
from surprise import SVDpp, SVD, NMF, SlopeOne, CoClustering, NormalPredictor
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from surprise.model_selection import GridSearchCV

In [5]:
# Load the preprocessed dataset
epinion_processed = pd.read_csv('epinion_processed_final.csv')
epinion_processed.head(3)

,item,paid,review,stars,time,user,cleaned_review,polarity,sentiment,textblob_polarity,textblob_sentiment
0,pr-Sony_KV_36FS12__Standard_Televisions,1199,well when i graduated from college it didnt ta...,5,1012262400,pyros7,well graduate college didnt take long realize ...,0.8090,5,0.312500,4
1,Minolta_QMS_PagePro_1250E_Printers,260,i got this printer from minolta as a warranty ...,4,1027296000,fgb59h,get printer minolta warranty replacement minol...,-0.0987,3,0.270833,4
2,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,1800,although i love my vaio especially the gorgeou...,2,1201305600,bucho_ky,although love vaio especially gorgeous display...,0.8883,5,0.240000,4


In [6]:
reader = Reader(rating_scale=(-1,1))

In [45]:
epinion_data = Dataset.load_from_df(epinion_processed[['user', 'item', 'polarity']], reader)
epinion_data.read_ratings

<bound method DatasetAutoFolds.read_ratings of <surprise.dataset.DatasetAutoFolds instance at 0x1a2969fdd0>>

In [48]:
# SPLITTING THE DATASET INTO TRAINING SET AND TESTING SET
trainset, testset = train_test_split(epinion_data, test_size=.25 , shuffle = True)

In [29]:

perf_results = []
for algo in [SVD(), SVDpp(), BaselineOnly(), CoClustering(), SlopeOne(), NormalPredictor()]:
    results = cross_validate(algo, epinion_data, measures=['RMSE'], cv=3, verbose=False)
    
    results_df = pd.DataFrame.from_dict(results).mean(axis=0)
    results_df = results_df.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    perf_results.append(results_df)
    
pd.DataFrame(perf_results).set_index('Algorithm').sort_values('test_rmse')

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


,fit_time,test_rmse,test_time
Algorithm,,,
BaselineOnly,1.202174,0.528181,0.966712
SVDpp,28.601935,0.529273,1.657370
SVD,12.584055,0.531011,1.197303
SlopeOne,66.775415,0.578354,3.626773
CoClustering,19.349045,0.630450,1.172580
NormalPredictor,0.511050,0.723703,0.897299


As SVD and SVDpp gave good rmse lets try to find best parameters of SVD and SVDpp using grid search

### Finding the best parameters using grid search

In [22]:
param_grid = {'n_epochs': [70, 90],  
              'lr_all': [ 0.002, 0.005, 0.01],
              'reg_all': [0.1,0.3]}

gssvd = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
gssvd.fit(epinion_data)
# best RMSE score
print(gssvd.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gssvd.best_params['rmse'])

0.5272813585819253
{'lr_all': 0.002, 'reg_all': 0.3, 'n_epochs': 70}


In [49]:
#algo = gssvd.best_estimator['rmse']
algo= SVD()
algo.fit(trainset)
predictions_svd = algo.test(testset)
accuracy.rmse(predictions_svd)

RMSE: 0.5299


0.5298502480177463

In [ ]:
param_grid = {'n_epochs': [70, 90],  
              'lr_all': [ 0.002, 0.005, 0.01],
              'reg_all': [0.1,0.3]}

gssvdpp = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=5)
gssvdpp.fit(epinion_data)
# best RMSE score
print(gssvdpp.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gssvdpp.best_params['rmse'])

In [ ]:
algo = gssvdpp.best_estimator['rmse']
algo.fit(trainset)
predictions_svdpp = algo.test(testset)
accuracy.rmse(predictions_svdpp)

In [16]:
algo = CoClustering()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.6363


0.636301396691609

### Top 3 recommendations for each user 

In [50]:
from collections import defaultdict
 
def top3_recommendations(predictions, topN = 3):
    recommendations = defaultdict(list)
    for user, item, polarity, est, _ in predictions:
        recommendations[user].append((item, est))
     
    for user, polarity in recommendations.items():
        polarity.sort(key = lambda x: x[1], reverse = True)
        recommendations[user] = polarity[:topN]
     
    return recommendations

    
top3_recommendations = top3_recommendations(predictions_svd)

# Recommending 3 items to each user:
i=0;
for user, polarity in top3_recommendations.items():
    print(user, [item for (item, _) in polarity])
    i=i+1;

(nan, ['JVC_HR_S3911U_4_Head_Super_VHS_VCR_VCRs'])
('darryld', ['auto_Make-2000_Mazda_Protege'])
('germanplumber', ['Polaroid_FLM_373B_Television'])
('seedenali', ['pr-Lexmark_X5150_Ink-jet_Printer'])
('s040606', ['pr-Initial_IDM-1731_DVD'])
('jwskatz', ['Mega_Bloks_Blok_Boat', 'kifm-Baby_Equipment-Strollers-All-Combi_Twin_Savvy_Stroller_7000_2', 'Mega_Bloks_Small_Maxi_Bag_New_Colors'])
('tsiklots', ['pr-Canon_PowerShot_A400_Digital_Camera'])
('jerebear44', ['SONICblue_RioVolt_SP90_Portable_Digital_CD_Player__MP3_Players_90260277'])
('gibssa1', ['2005_Nissan_Altima'])
('dem45133', ['auto_Make-1994_Chrysler_Concorde', 'auto_Make-1994_Dodge_Ram_1500'])
('luckymen', ['Black_Decker_TRO651_Toast_R_Oven_Toaster_Oven_with_Convection_Cooking'])
('knitsnbytes', ['Braun_Multiquick_Minipimer_Blender_MR_400_HC__Blenders_MR400HC', 'George_Foreman_GRP4P', 'pr-Omega_8002_Food_Processor'])
('jcdenton', ['cmhd-Desktops-All-Apple_iMac_DV_plus'])
('ihatemiele', ['Miele_Navitronic_H4890BP2_Oven'])
('jkay7

('tracker464', ['auto_Make-1996_Ford_Ranger'])
('PEDIT', ['auto_Make-2003_Kia_Rio', 'pr-Car_Stereos_Changers_Kenwood_DPX_5010_Radio_CD_Cassette_Player_DPX_5010', 'cmsw-ISP-All-AOL'])
('jumpshot', ['pr-SanDisk_SDMX1512A18_512_MB_MP3_Player'])
('chooseChrist', ['pr-Panasonic_RR-US350_Voice_Recorder', 'cmsw-ISP-All-AT_T_WorldNet'])
('lord12334', ['samsung-galaxy-s-iii-cell-phone'])
('leomeister', ['auto_Make-2003_Pontiac_Vibe', 'pr-Canon_PowerShot_SD300_Digital_IXUS_40_Digital_Camera', 'pr-Oreck_Super_Air_7_Air_Purifier'])
('andavid', ['auto_Make-2002_Mazda_MPV'])
('Joanne710', ['Mattel_Happy_Family_Midge_and_Baby_32723654', 'Play_Along_Lisa_Frank_Fab_Friends_11_Mara_Doll_with_Exclusive_Pet'])
('hawkeye_1', ['510923_Intel_Pocket_Concert_Audio_Player'])
('xcountrygirl', ['MGA_Entertainment_Bratz_Jade'])
('steviesharon', ['hmgd-Large_Appliances-All-Washing_Machines-Kenmore_3_0_cu__ft__Super_Capacity_Top_Load_Washer_with_6_Cycles_and_Options_22442'])
('thebigstick', ['Chatter_Phone_32724160'

('themommy', ['kifm-Baby_Equipment-Car_Seats-All-Eddie_Bauer_Three_Car_Seats_in_One'])
('jegesmedve', ['2000_Yamaha_XT225'])
('dlaz', ['pr-Canon_PowerShot_A80_Digital_Camera'])
('greycatpaw', ['Toshiba_M45_S165_Satellite_Notebook_PC_PSM43U00S00F_PC_Notebook', 'Motorola_V400_Quad_Band_Phone'])
('odoneye', ['pr-Asko_T761_Standard_Dryer'])
('tamara5', ['Tefal_TT7061002_Toaster'])
('p00ch', ['pr-2006_Mazda_5'])
('jemille', ['auto_Make-2003_Mazda_MX_5_Miata'])
('ceo333', ['Sony_PCG_SRX87_PIII_850MHz___256MB___10_4__TFT___Win_XP_Home_Edition_9___Notebook_Computer__Notebooks_PCG_SRX87'])
('gretagrrrl', ['Lawrence'])
('liforily7', ['hmgd-Large_Appliances-All-Washing_Machines-Maytag_Atlantis_Washer_MAV6200A'])
('columbia_rider', ['2000_Honda_Shadow_Spirit'])
('snpmurray', ['pr-Emerson_EWC20D3_20_in_TV_DVD_Combo'])
('supercooper', ['hmgd-Moving_Services-All-ABF_U_Pack_Moving_Moving_Service'])
('ron77707', ['pr-Digital_Cameras-Sony_CD_Mavica_CD500_Digital_Camera'])
('pande', ['well-Personal-Air-P

('nmarx1', ['hmgd-Large_Appliances-All-Ranges-KitchenAid_Biscuit_30_in_Gas_Self-Clean_Convection_Oven_w_Sealed_Burners_Electronic_Controls_KGRT607HBT'])
('jharrisond', ['S0311586_Compaq_iPAQ_H3630'])
('ckwilinski', ['General_Reviews_of_T_Mobile_Customer_Service'])
('doufuss', ['pr-KitchenAid_KERC507_Kitchen_Range'])
('jsharley', ['pr-Kenmore_23913_Vacuum'])
('SwtPsMama', ['Rub_A_Dub_Draw_In_The_Tub_by_Alex_Toys'])
('kopmama', ['pr-Cribs_Sorelle_Rita_Bella_15936'])
('t2iv', ['pr-Viking_VGIC365-6B_Kitchen_Range'])
('hestein', ['Samsung_TV_and_Video_DVD_Recorder_036725601215'])
('kugluga', ['pr-Digital_Cameras-Canon_PowerShot_A70'])
('groupeone', ['Fisher_and_Paykel_AeroTech_OS302_Oven'])
('tiffani_0602', ['Baby_Playzone_Stand_Up_Ball_Blast'])
('mangotree', ['Fisher_Price_PowerTouch_Learning_System'])
('lilviv', ['RIM_Research_In_Motion_BB8900CURVE_Curve_8900_Unlocked_Cell_Phone'])
('raymcgill', ['auto_Make-1998_Ford_Expedition'])
('jmbuck', ['hmgd-Moving_Services-All-Dependable_Auto_Ship

('gil_ramsey', ['pr-Infinity_Reference_REF_6802CF_Car-Multi-Element_Speaker'])
('iunhappy', ['Maytag_27_Cubic_foot_Wide_Side_By_Side_Refrigerator_MZD2766WH_Refrigerators'])
('rajandleena', ['auto_Make-2001_Honda_Odyssey'])
('markerfumes93', ['Handspring_Treo_300_PDAs'])
('Stockman43', ['Motorola_Timeport_P8767_Phone_Cellular_Phone'])
('monk-ey-boy', ['hmgd-Vacuum_Cleaners-Dirt_Devil-Dirt_Devil_Ultra_Hand_Vac'])
('dvpdpv11', ['apple-iphone-4s-mobile-phone'])
('icollect', ['auto_Make-2003_Dodge_Ram_1500_Truck'])
('kjhall75', ['Crane_Crane_Adorable_Frog_Humidifier_Humidifier_2'])
('enchantingquen', ['pr-Creative_Labs_Nomad_Jukebox_Zen_Xtra_30GB_MP3_Player'])
('ggpro', ['pr-Target_CTO6300_Toaster_Oven'])
('cll8857', ['pr-Fisher_and_Paykel_GWL11_Washing_Machine', 'Floam_32723451'])
('yolj', ['elec_Car_Stereo_In_Dash_Stereo-With_CD-Pioneer_PioneerDEHProductFamilyily-Pioneer_DEH-P1000'])
('ddengler', ['auto_Make-2003_Chevrolet_Impala'])
('kbacal', ['Dell_Inspiron_E1405_Single_Core_Notebook_Co

...

('blaed-', ['elec_Home_Speakers-Stereo_Paradigm_Monitor-Paradigm_Monitor_9'])
('scibba', ['Leap_Frog_Mind_Station_Learning_Toy_Expansion_System'])
('epeejerm', ['Nokia_8260_AMPS_D_AMPS_Cellular_Phone_Cellular_Phone'])
('pretty_jules', ['Cosco_Infant_Car_Seat_with_Base_Car_Seats'])
('jessica1520', ['kifm-Camps-All-Camp_La_Jolla'])
('roleku', ['pr-Russell_Hobbs_2_Toaster', 'pr-Kyocera_Mita_7135'])
('jasonburns', ['Casio_EXILIM_ZOOM_EX_Z70_Digital_Camera'])
('bruce2402', ['pr-3Com_OFFICECONNECT_WRLS_108MBPS_11G_ACCESS_POINT_WITH_POE_3CRGPOE10075US'])
('sheener', ['pr-Chicco_Tuscany_Double_Umbrella_Stroller'])
('romendoz', ['SimpleTech_SimpleShare_STI_NAS_250_STINAS250'])
('staceye', ['Bertini_Shuttle_M5_Steerable_Pram_Stroller_421_M5_Strollers'])
('go-bucks', ['Syntax_Olevia_2_Series_1080p_65_LCD_TV_265TFHD_1'])
('marksreardon', ['kifm-Camps-All-Supercamp'])
('pumper', ['Hewlett_Packard_C8901A_ABA_629663'])
('tim1967', ['auto_Make-2002_GMC_Yukon_Denali'])
('pminthecity', ['pr-GE_Upright_B

('diablote', ['elec-Photo-Cameras-All-Casio_QV_3000EX'])
('therealgr8one', ['9_6V_Radio_Control_Flashback'])
('zoomn5', ['2007_Mazda_MX_5_Miata'])
('joefossil', ['Apple_AC_Power_Adapter_for_iPod__AC_Adapters_M8636LL_A'])
('benminkoff', ['auto_Make-2001_Honda_Odyssey'])
('gms049', ['597415_Sony_SSM_B300H_Bookshelf_Speaker'])
('tm59', ['pr-DVD_Players-CyberHome_CH-LDV_712_Portable_DVD_Player'])
('ryanblast28', ['apple-iphone-5-mobile-phone-131786819'])
('jotalance', ['pr-Apple_iBOOK_m9426lla_Mac_Notebook'])
('otoole77', ['auto_Make-1993_Buick_Century', 'well-Personal-Electric_Razors-All-Braun_Flex_Integral_ultra_speed_6520'])
('gckunkle', ['pr-Admiral_DWD1500A_Dishwasher'])
('hnybnchsofoats', ['auto_Make-2002_Mazda_Miata'])
('mediak', ['pr-Eureka_Enviro_Steamer_300_Vacuum'])
('charolien', ['pr-Apple_iPod_Fourth_Gen_20_GB_MAC_PC-M9282LL_A_MP3_Player', 'pr-Laptops-Dell_Inspiron_1100_Series_Notebook_Computer'])
('adambomb_12', ['hmgd-Moving_Services-All-ABF_U_Pack_Moving_Moving_Service'])
(

('daslayer321', ['cmhd-Notebooks-All-Apple_Powerbook_G3', 'ThinkPad_T20_2647_S0185337'])
('sskein', ['Hug-A-Bub'])
('vkidsplay6', ['auto_Make-2001_Ford_Expedition'])
('mannishboy24', ['auto_Make-1995_Plymouth_Acclaim'])
('blackii', ['Concord_Camera_EyeQ_Duo_Digital_Camera__Camera_EyeQ_Duo'])
('josemalone', ['International_Playthings_Little_Doctor_Kit_G02021'])
('rob98374', ['pr-Mitsubishi_WL-82913_82_in_Rear_Projection_HDTV_Television'])
('miheala', ['Kodak_EasyShare_Z650_Digital_Camera'])
('webguy', ['cmhd-Printers-All-Epson_Stylus_Color_777'])
('debbiebhome', ['pr-Kenmore_Kenmore_3_2_cu_ft_King_Size_Capacity_Top_Load_Washer_Washing_Machine'])
('ginylee', ['auto_Make-2002_Chrysler_PT_Cruiser'])
('candybowen', ['dyson-dc35-multi-floor-vacuum'])
('jostlin', ['Evenflo_Triumph_Deluxe_Diamondback', 'Zooper_Zydeco_Z344_Stroller'])
('cjoslin', ['pr-Lexmark_Optra_W810dn_Laser_Printer'])
('wendychurch', ['auto_Make-2002_Buick_Rendezvous'])
('Leanne_Engle', ['Apple_iMac_PowerPC_G3_500MHz___20GB

('robw64', ['auto_Make-2001_Hyundai_XG300'])
('wickedchisel', ['cmsw-ISP-All-gisol_com'])
('ttheobal', ['Barricade_g_SMCWBR14T_G_Router_SMCWBR14TG'])
('jayyy9', ['Miele_S_5980_SEB_217_Vacuum', 'Dyson_DC07BT_Upright_Vacuum'])
('robert9803', ['Kodak_EasyShare_LS443_Digital_Camera_Digital_Cameras'])
('nottowing', ['pr-Panasonic_PT-51HX41_51_in_Rear_Projection_Television'])
('kymy', ['Kids_II_Bright_Starts_Tropical_Smiles_Around_We_Go', 'Kolcraft_Jeep_Liberty_Limited_Energy_Stroller', 'pr-Hewlett_Packard_HP_Photosmart_8450_Photo_Printer_Printer'])
('jeffersmom', ['Fantom_BPSRLC51_Lightning_Canister_Vacuum_Cleaner_Vacuum_Cleaners', 'Fisher_Price_Rescue_Heroes_Dune_Buggy'])
('genaandmark', ['pr-GE_32GT740__Standard_Televisions'])
('missFee', ['SuperGlow_Electronic_Kitchen'])
('tester411', ['kifm-Baby_Equipment-Car_Seats-All-Safeline_Sit_N_Stroll_Nitelite_Navy_Car_Seat_02240NN_1'])
('milmac85', ['auto_Make-1996_Saturn_SL'])
('dw06', ['auto_Make-2002_Nissan_Altima'])
('nochumplulv', ['Warhamme

('sweetspirit', ['Canon_CPK_A10_Personal_Digital_Photography_Kit__Printers_CPKA10KIT'])
('usonianm', ['APPLE_IMAC_G5_2_0_250GB_512MB_SD_20_M9845LLA_Mac_Desktop'])
('nerdiedude', ['Intel_Entry_Storage_System_SS4000_E_NAS_server_SS4000ENA'])
('consumer1452', ['pr-Panasonic_NN-T964SF_Microwave_Oven'])
('dmd2157', ['pr-Sony_Cyber-Shot_DSC-U60_Digital_Camera'])
('dstoer', ['hmgd-Lawn_and_Garden-Outdoor_Cooking-Grills_and_Smokers-All-Weber_One_Touch_Gold_Grill'])
('colbeck', ['Toshiba_SD5700_624630'])
('vrythmax', ['pr-Hewlett_Packard_HP_Pavilion_F1503_15_in_Flat_Panel_TFT_Monitor'])
('judagar', ['elec_Cameras-Point_And_Shoot-Contax_ContaxTProductFamilyily-Contax_T2', 'auto_Make-2000_Volkswagen_Beetle'])
('pde99', ['Motorola_v2397_Cellular_Phone'])
('tzbfwt', ['Black_Decker_CTO6301_Toaster_Oven_with_Convection_Cooking'])
('greno', ['Compaq_Presario_700UK_PIII_900MHz___20GB___128MB___DVD___14_1__TFT___WinXP___Notebook_Computer__Notebooks_470024_192'])
('jetaz', ['Hewlett_Packard_HP_Hewlett_Pa

### https://surprise.readthedocs.io/en/stable/getting_started.html